# Machine learning, a tutorial, part III



## 3-1 Revist tree-based models

In [1]:
## import packages
import pandas as pd
import numpy as np

## some setting for better reading experience
from IPython.display import display
pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.4f' % x)

##  
randomState = 8

In [2]:
## read data 
melb = pd.read_csv("data/melb_training_data.csv").sample(frac=1, random_state=randomState).reset_index(drop=True)
melb.drop(columns = ["Unnamed: 0"], inplace=True)


In [3]:
## select the features
features_in_model = ["Distance", "Rooms", "YearBuilt", "BuildingArea","Landsize"]
X = melb[features_in_model]

## set the target
y = melb["Price"]

## Train / test split
X_train = X[:9000]
y_train = y[:9000]
X_test = X[9000:]
y_test = y[9000:]

In [4]:
## Random forest
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators = 300, 
                                bootstrap = True,
                                 oob_score = True,
                                 n_jobs = 3,
                                 random_state = randomState)

print("With the no-feature-engineering data (missing value only):")
rf_model.fit(X_train.fillna(-1), y_train)

print("R^2 on training data = {:.4f}".format(rf_model.score(X_train.fillna(-1), 
                                                            y_train)))
print("R^2 on test data = {:.4f}".format(rf_model.score(X_test.fillna(-1), 
                                                        y_test)))

With the no-feature-engineering data (missing value only):
R^2 on training data = 0.9404
R^2 on test data = 0.5802


In [5]:
## GBDT
from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor(n_estimators = 300, 
                                     subsample = 0.7,
                                    n_iter_no_change = 10,
                                     random_state = randomState)

print("With the no-feature-engineering data (missing value only):")
gb_model.fit(X_train.fillna(-1), y_train)

print("R^2 on training data = {:.4f}".format(gb_model.score(X_train.fillna(-1), 
                                                            y_train)))
print("R^2 on test data = {:.4f}".format(gb_model.score(X_test.fillna(-1), 
                                                        y_test)))


With the no-feature-engineering data (missing value only):
R^2 on training data = 0.6922
R^2 on test data = 0.6052


In [6]:
## Take look the first row in test data 

one_line_test = X_test.head(1).fillna(-1)

one_line_test

,Distance,Rooms,YearBuilt,BuildingArea,Landsize
9000,6.4000,2,1990.0000,89.0000,312.0000


In [7]:
## And the predicted values from random forest and GBDT
print("Predicton from RF = {:.4f}".format(rf_model.predict(one_line_test)[0]))
print("Predicton from GBDT = {:.4f}".format(gb_model.predict(one_line_test)[0]))
print("True value = {:.4f}".format(y_test.values[0]))



Predicton from RF = 728720.0000
Predicton from GBDT = 637323.0770
True value = 468000.0000


In [8]:
## Take the first decision tree in the random forest I've trained
rf_model.estimators_[0].predict(one_line_test)


array([612500.])

In [9]:
## Take all the prediction results in random forest
rf_results = list()

for i in range(len(rf_model.estimators_)):
    rf_results.append(rf_model.estimators_[i].predict(one_line_test))

## The RF take the average of the results from all the trees
np.mean(rf_results)


728720.0

In [10]:
## On the other hand, GBDT add all results up
gb_results = list()

for i in range(len(gb_model.estimators_)):
    gb_results.append(gb_model.estimators_[i][0].predict(one_line_test))

## gb_model.init_ is the mean(y) from training data as the 0-th prediction
## the learning_rate controls how fast the results are added up
gb_model.init_.predict(one_line_test) + np.sum(gb_results)*gb_model.learning_rate 

array([637323.07695403])

## 3-2 Model evaluation

In [11]:
## If I have only few models, it's ok to use test R2 for choosing the best model

print("Random Forest: R^2 on test data = {:.4f}".format(rf_model.score(X_test.fillna(-1), 
                                                        y_test)))

print("GBDT: R^2 on test data = {:.4f}".format(gb_model.score(X_test.fillna(-1), 
                                                        y_test)))

Random Forest: R^2 on test data = 0.5802
GBDT: R^2 on test data = 0.6052


In [12]:
## I want to find a good set of hyperparameters
## Take GBDT for example, the hyperparameters to choose is learning_rate & min_samples_leaf

learning_rates = np.arange(0.01, 0.3, 0.06)
min_samples_leafs = np.arange(1, 20, 4)

## 5 values for both => number of models = 25
## this is grid search
print(learning_rates)
print(min_samples_leafs)


[0.01 0.07 0.13 0.19 0.25]
[ 1  5  9 13 17]


In [13]:
## try all the combinations

hyper_searching_results = dict()

for lr in learning_rates:
    for mnl in min_samples_leafs:
        #print(lr, mnl)
        gb_model = GradientBoostingRegressor(n_estimators = 300, 
                                     subsample = 0.7,
                                    n_iter_no_change = 10,
                                     learning_rate = lr, 
                                     min_samples_leaf = mnl,
                                     random_state = randomState)

        gb_model.fit(X_train.fillna(-1), y_train)
        R2_test = gb_model.score(X_test.fillna(-1), y_test)
        hyper_searching_results[(lr, mnl)] = R2_test
        
hyper_searching_results

{(0.01, 1): 0.5256142302658419,
 (0.01, 5): 0.5250243798008423,
 (0.01, 9): 0.5236738947853083,
 (0.01, 13): 0.5229067341597174,
 (0.01, 17): 0.5228485455369929,
 (0.06999999999999999, 1): 0.6045845572552284,
 (0.06999999999999999, 5): 0.5693352731126812,
 (0.06999999999999999, 9): 0.5696395016397571,
 (0.06999999999999999, 13): 0.5975864041706282,
 (0.06999999999999999, 17): 0.5940958065635806,
 (0.12999999999999998, 1): 0.5782171638256237,
 (0.12999999999999998, 5): 0.5668037781274522,
 (0.12999999999999998, 9): 0.5758478888745222,
 (0.12999999999999998, 13): 0.5930712179623087,
 (0.12999999999999998, 17): 0.5977175520732401,
 (0.18999999999999997, 1): 0.6002485635061221,
 (0.18999999999999997, 5): 0.5586668459807924,
 (0.18999999999999997, 9): 0.6095951724817985,
 (0.18999999999999997, 13): 0.5877448245871291,
 (0.18999999999999997, 17): 0.6119960338501851,
 (0.24999999999999997, 1): 0.6085056003082734,
 (0.24999999999999997, 5): 0.6018058035253546,
 (0.24999999999999997, 9): 0.5982

In [14]:
## the winner goes to ......
max(hyper_searching_results, key=hyper_searching_results.get)

(0.18999999999999997, 17)

### 3-2-1 Cross Validation & Hyperparameter tuning
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV

In [ ]:
## in many applications, we don't abuse test set like that
## we use cross validation to replace multiple evaluations on test set

## to do CV, there is no need to write multiple loops all by myself
from sklearn.model_selection import GridSearchCV

parameters_to_search = {'learning_rate': learning_rates, 
              'min_samples_leaf': min_samples_leafs}

gb_model = GradientBoostingRegressor(n_estimators = 300, 
                                     subsample = 0.7,
                                    n_iter_no_change = 10,
                                     random_state = randomState)

gb_model_CV = GridSearchCV(gb_model, parameters_to_search, cv=5)
gb_model_CV.fit(X_train.fillna(-1), y_train)




In [ ]:
## the gridcv module run the models and save the results for us
gb_model_CV.cv_results_

In [110]:
## the mean of 5-folds test(not true test) R2
gb_model_CV.cv_results_["mean_test_score"]

array([0.53843195, 0.53889237, 0.53819574, 0.5381507 , 0.53753945,
       0.61588835, 0.61846628, 0.6162798 , 0.61306669, 0.61027594,
       0.61261881, 0.61777265, 0.61322641, 0.61834923, 0.61143677,
       0.6102079 , 0.62499701, 0.60372491, 0.61588563, 0.60891797,
       0.60834103, 0.6177238 , 0.61079874, 0.60410298, 0.61357182])

In [109]:
## the best one is learning_rate=0.18999999999999997, min_samples_leaf=5
gb_model_CV.best_estimator_

GradientBoostingRegressor(learning_rate=0.18999999999999997, min_samples_leaf=5,
                          n_estimators=300, n_iter_no_change=10, random_state=8,
                          subsample=0.7)

## 3-3 Learn from models

### 3-3-1 Feature importance
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html?highlight=randomforestregressor#sklearn.ensemble.RandomForestRegressor.feature_importances_

In [111]:
## It's easy to get feature inportance from tree-based models
## the values is normalized variation reduction
## sum of them is 1

rf_model.feature_importances_

array([0.3028007 , 0.23719954, 0.0910567 , 0.14050044, 0.22844263])

In [112]:
## The order corresponds to the features you put in the model
features_in_model

## so distance is important when predicting house price
## but is this equivalent to the answer from linear model?

['Distance', 'Rooms', 'YearBuilt', 'BuildingArea', 'Landsize']

In [180]:
## for linear models

from sklearn.linear_model import LinearRegression

X_train_lr = X_train.copy()
X_test_lr = X_test.copy()

## missing values
mean_YearBuilt = X_train.YearBuilt.mean()
mean_BuildingArea = X_train.BuildingArea.mean()

X_train_lr["YearBuilt"] = X_train["YearBuilt"].fillna(mean_YearBuilt)
X_train_lr["BuildingArea"] = X_train["BuildingArea"].fillna(mean_BuildingArea)

X_test_lr["YearBuilt"] = X_test_lr["YearBuilt"].fillna(mean_YearBuilt)
X_test_lr["BuildingArea"] = X_test_lr["BuildingArea"].fillna(mean_BuildingArea)


## train without any transformation
lr_model = LinearRegression(fit_intercept=True)

## train the Linear regression model
lr_model.fit(X_train_lr, y_train)

print("Linear : R^2 on test data = {:.4f}".format(lr_model.score(X_test_lr, 
                                                        y_test)))

Linear : R^2 on test data = 0.3956


In [181]:
## the coefficents
for i in range(len(features_in_model)):
    print("Coefficient for {} = {:.4f}".format(features_in_model[i], lr_model.coef_[i]))



Coefficient for Distance = -32130.3773
Coefficient for Rooms = 377407.7353
Coefficient for YearBuilt = -4158.6728
Coefficient for BuildingArea = 45775923.3946
Coefficient for Landsize = 3.8707


In [182]:
## if the unit of some variables changes, the corresponding coefficient also alters

X_train_lr["BuildingArea"] = X_train_lr["BuildingArea"] / 1000
X_test_lr["BuildingArea"] = X_test_lr["BuildingArea"] / 1000

## train without any transformation
lr_model = LinearRegression(fit_intercept=True)

## train the Linear regression model
lr_model.fit(X_train_lr, y_train)
print("Linear : R^2 on test data = {:.4f}".format(lr_model.score(X_test_lr, 
                                                        y_test)))

## the coefficents remain the same except the one for BuildingArea
## the one for BuildingArea is 1000x larger
for i in range(len(features_in_model)):
    print("Coefficient for {} = {:.4f}".format(features_in_model[i], lr_model.coef_[i]))


Linear : R^2 on test data = 0.3956
Coefficient for Distance = -32130.3773
Coefficient for Rooms = 377407.7353
Coefficient for YearBuilt = -4158.6728
Coefficient for BuildingArea = 45775923394.7389
Coefficient for Landsize = 3.8707


In [194]:
## Standardization
## https://scikit-learn.org/stable/modules/preprocessing.html

## convert x to (x - mean(x)) / sd(x)
## the "unit" of x is "one standard deviation from mean"
## so this can be compared between all features
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_train_lr)
X_train_lr = scaler.transform(X_train_lr)
X_test_lr = scaler.transform(X_test_lr)

## train the Linear regression model
lr_model = LinearRegression(fit_intercept=True)
lr_model.fit(X_train_lr, y_train)
print("Linear : R^2 on test data = {:.4f}".format(lr_model.score(X_test_lr, 
                                                        y_test)))

## Now the meaning of coefficients is "the delta of y by 1 standard deviation of x from mean"
## compare the result to feature importance derived from random forest
for i in range(len(features_in_model)):
    print("Coefficient for {} = {:.4f}".format(features_in_model[i], lr_model.coef_[i]))





Linear : R^2 on test data = 0.3956
Coefficient for Distance = -188170.9616
Coefficient for Rooms = 361911.0437
Coefficient for YearBuilt = -117549.6360
Coefficient for BuildingArea = 21882.1181
Coefficient for Landsize = 18586.9919


### 3-3-2 Feature Selection

In [ ]:
## For tree-based models, you can just drop the features with low importance values
## For linear models, you can drop features with the results from tree-base models

## unlike the tree-based model, drops "useless features" from linear models affects results
## you can treat "feature combination" just like "hyperparameter combination" in a CV manner
## however, it's not a "Grid search" task
## you should use sklearn.model_selection.cross_validate to do this
## the grammar are similar

### 3-4 What is a good model (for you)?